In [ ]:
%matplotlib widget
import os
import datetime
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
today=datetime.date.today()

In [ ]:
import ipywidgets as widgets
import ipympl
from IPython.display import clear_output

In [ ]:
def gaussian_norm(x,xc,fwhm,area):
    norm = 1./fwhm/np.sqrt(np.pi/4/np.log(2))
    return area*norm*np.exp(-4*np.log(2)*(x-xc)**2/(fwhm)**2)

def fitpeak(data):
    center = np.argmax(data)
    xdata = np.arange(len(data))
    popt,pcov = curve_fit(gaussian_norm, xdata, data, p0=[center,10.,np.max(data)])
    mu = popt[0]
    sigma = popt[1]
    amp = popt[2]
    return mu,sigma,amp

def getdata(scannumber):
    global mypath
    global base
    
    if scannumber < 10:
        filename = mypath+'\\'+base+"_"+"000"+str(scannumber)
    elif scannumber < 100:
        filename = mypath+'\\'+base+"_"+"00"+str(scannumber)
    elif scannumber < 1000:    
        filename = mypath+'\\'+base+"_"+"0"+str(scannumber)
    else:    
        filename = mypath+base+"_"+str(scannumber)

    f1 = h5py.File(filename+"_d1.h5", 'r')
    f2 = h5py.File(filename+"_d2.h5", 'r')
    f3 = h5py.File(filename+"_d3.h5", 'r')

    ccd1 = np.array(f1['entry']['analysis']['spectrum'][()])
    ccd2 = np.array(f2['entry']['analysis']['spectrum'][()])
    ccd3 = np.array(f3['entry']['analysis']['spectrum'][()])

    energy = round(np.mean(f3['entry']['instrument']['NDAttributes']['PhotonEnergy'][()]),2)
    
    return energy,[ccd1,ccd2,ccd3]

def getdatafit(scannumber):
    
    energy,[ccd1,ccd2,ccd3] = getdata(scannumber)
    
    mu1,sig1,amp1 = fitpeak(ccd1)
    mu2,sig2,amp2 = fitpeak(ccd2)
    mu3,sig3,amp3 = fitpeak(ccd3)
    
    return energy,[mu1,sig1,amp1,mu2,sig2,amp2,mu3,sig3,amp3]

In [ ]:
def plot_raw():
    fig = plt.figure()

    for i,s in enumerate(scans):
        energy,[ccd1,ccd2,ccd3] = getdata(s)
        offset = 200
        xdata = np.arange(len(ccd1))
        plt.plot(xdata,ccd1+offset*i,marker='.',color = "r")
        plt.plot(xdata,ccd2+offset*i,marker='.', color = "g")
        plt.plot(xdata,ccd3+offset*i,marker='.', color = "b")
        en,[xc1,fwhm1,area1,xc2,fwhm2,area2,xc3,fwhm3,area3]= getdatafit(s)
        plt.plot(xdata,gaussian_norm(xdata,xc1,fwhm1,area1)+offset*i,'-',color = "black")
        plt.plot(xdata,gaussian_norm(xdata,xc2,fwhm2,area2)+offset*i,'-',color = "black")
        plt.plot(xdata,gaussian_norm(xdata,xc3,fwhm3,area3)+offset*i,'-',color = "black")
        
    plt.xlabel('subpixel')
    plt.ylabel('Counts/1 min')
    # plt.savefig(str(today)+'_'+base+'_'+'drift.pdf',dpi=600,format='pdf')
    
    plt.show()


In [ ]:
def plot_center():
    fig = plt.figure()

    for i,s in enumerate(scans):

        en,[xc1,fwhm1,area1,xc2,fwhm2,area2,xc3,fwhm3,area3]= getdatafit(s)
        plt.plot(en,fwhm1/4,'o',color = "black")
        plt.plot(en,fwhm2/4,'o',color = "red")
        plt.plot(en,fwhm3/4,'o',color = "blue")
        
    plt.xlabel('Photon Energy (eV)')
    plt.ylabel('FWHM (pixel)')
    # plt.savefig(str(today)+'_'+base+'_'+'FWHM.pdf',dpi=600,format='pdf')
    
    plt.show()

In [ ]:
def plot_slope():
    fwhm1 =np.array([])
    fwhm2 = np.array([])
    fwhm3 = np.array([])
    center1 = np.array([])
    center2 = np.array([])
    center3 = np.array([])
    energy = np.array([])

    for i,s in enumerate(scans):
        en,[xc1,gamma1,area1,xc2,gamma2,area2,xc3,gamma3,area3]= getdatafit(s)
        energy = np.append(energy,en)
        fwhm1 = np.append(fwhm1,gamma1)
        center1 = np.append(center1,xc1)
        fwhm2 = np.append(fwhm2,gamma2)
        center2 = np.append(center2,xc2)
        fwhm3 = np.append(fwhm3,gamma3)
        center3 = np.append(center3,xc3)

    [myslope1,myinter1] = np.polyfit(energy,center1,1)
    [myslope2,myinter2] = np.polyfit(energy,center2,1)
    [myslope3,myinter3] = np.polyfit(energy,center3,1)
    energypersubpixel = 1/np.mean([myslope1,myslope2,myslope3])
    xplot = np.linspace(energy[0],energy[-1],100)
    plt.figure()
    plt.errorbar(energy,center1,yerr=fwhm1,xerr=None,linewidth=0,marker = "o")
    plt.errorbar(energy,center2,yerr=fwhm2,xerr=None,linewidth=0,marker = "o")
    plt.errorbar(energy,center3,yerr=fwhm3,xerr=None,linewidth=0,marker = "o")
    plt.xlabel('Photon Energy (eV)')
    plt.ylabel('Elastic position (subpixel)')
    plt.plot(xplot,myslope1*xplot+myinter1, color = 'black')
    plt.plot(xplot,myslope2*xplot+myinter2, color = 'black')
    plt.plot(xplot,myslope3*xplot+myinter3, color = 'black')
    plt.annotate('Energy dispersion = '+str(round(energypersubpixel*1000,3))+' meV/subpixel',xy = (energy[0],center1[-1]))
    plt.annotate('Energy resolution = '+str(round(np.mean([fwhm1,fwhm2,fwhm3])*energypersubpixel*1000,3))+' meV',xy = (energy[0],center1[-2]))
    plt.savefig(str(today)+'_'+base+'_'+'slope.pdf',dpi=600,format='png')
    plt.show()

In [ ]:
def do_plot():
    os.chdir(mypath)
    plot_raw()
    plot_center()
    plot_slope()

In [ ]:
path = widgets.Text(
    value=r'X:\\RIXS\\Asmara',
    placeholder='Parent Path',
    description='Project Path:',
    disabled=False
)
display(path)

atom = widgets.Text(
    value='O',
    placeholder='Element',
    description='Prefix:',
    disabled=False
)
display(atom)

scan = widgets.Text(
    value='1,2,3',
    placeholder='1,2,3',
    description='Scan:',
    disabled=False
)
display(scan)

plot = widgets.Button(description="Plot")
output = widgets.Output()

display(plot,output)

In [ ]:
def on_button_clicked(b):
    output.clear_output()

    with output:
        global mypath,base,scans
        mypath = path.value
        base = atom.value
        scans = tuple(map(int,scan.value.split(',')))
        do_plot()

plot.on_click(on_button_clicked)